In [9]:
import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
import torch.optim as optim
from PIL import Image
import numpy as np

from pathlib import Path

from matplotlib import pyplot as plt
%matplotlib inline

import shutil

from pathlib import Path
torch.cuda.device(0)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [10]:
crash_frame_idx = 5454
end_frame = 19050

In [11]:
net = torchvision.models.resnet18(pretrained=False)
net = net.to(device)
num_fc_input = net.fc.in_features
print(num_fc_input)
net.fc = nn.Linear(num_fc_input, 2).cuda()
#net.fc = 
#net.fc = nn.Sequential(nn.Linear(num_fc_input, 1).to(device),
#                      nn.Sigmoid())


512


In [12]:
# load trained weight
model_name = "sample"
weight_path = f'../models/{model_name}.pt'
net.load_state_dict(torch.load(weight_path))

<All keys matched successfully>

In [13]:
# image preprocess
transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])

In [14]:
# dataset class

image_dir = Path('../data/frames')

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self):
        super(CustomDataset, self).__init__()

        # 前処理クラスを受け取る
        self.transform = transform

        # ラベルの用意
        img_list = []
        label_list = []

        pos_num = 0
        neg_num = 0

        for idx in range(crash_frame_idx, end_frame):
            img_idx = str(idx).zfill(5)
            image_path = image_dir / ('frame_' + img_idx + '.jpg')
            assert image_path.exists()
            img_list.append(str(image_path))
            label_list.append(int(idx))
        
        self.images = img_list
        self.labels = label_list
        self.num_balance = [float(neg_num), float(pos_num)]

    def __getitem__(self, index):
        image = self.images[index]
        label = self.labels[index]

        with open(image, 'rb') as f:
            image = Image.open(f)
            image = image.convert('RGB')
        
        # 前処理を入れる
        if self.transform is not None:
            image = self.transform(image)

        return image, label

    def __len__(self):
        return len(self.images)

c = CustomDataset()
c.images[0:3]

['../data/frames/frame_05454.jpg',
 '../data/frames/frame_05455.jpg',
 '../data/frames/frame_05456.jpg']

In [15]:
custom_dataset = CustomDataset()
test_loader = torch.utils.data.DataLoader(dataset=custom_dataset,
                                            batch_size=1,
                                            )

In [17]:
%%time
output_path = Path(f"../results/{model_name}")
output_path.mkdir(exist_ok=True, parents=True)
output_path = str(output_path)

# if an image whose estimated value is over threshold, recognized as containing flagman 
threshold = 0.9

softmax = torch.nn.Softmax(dim=1)


im_list1 = []
net.eval()
with torch.no_grad():
    for i, (images, labels) in enumerate(test_loader):
        images, labels =images.to(device), labels.to(device)
        outputs = net(images)
        prob = (softmax(outputs)).cpu().numpy().tolist()
        pos_prob = round(float(prob[0][1]), 4)

        if pos_prob > threshold:
            x = int(labels)
            print(x, pos_prob)
            image_path = image_dir / ('frame_' + str(x).zfill(5) + '.jpg')
            im_list1.append(x)
            shutil.copy(str(image_path), output_path)


5507 0.9407
5538 0.938
5542 0.9976
5579 0.9613
5582 0.9445
5634 0.9573
5700 0.972
5765 0.9892
5792 0.9293
5863 0.9134
5869 0.9016
5933 0.9972
5994 0.9832
5996 0.9662
6020 0.9628
6094 0.9215
6096 0.9642
6121 1.0
6128 0.9365
6142 0.9202
6150 0.9678
6152 0.9856
6170 0.9357
6172 0.9932
6194 0.9717
6198 0.9792
6212 0.9076
6306 0.9636
6326 0.9692
6344 0.9402
6355 1.0
6378 0.9879
6383 0.9635
6415 0.9625
6436 0.9934
6438 0.993
6445 0.9869
6473 0.9117
6509 0.965
6529 0.9621
6563 0.9866
6577 0.9211
6610 0.9913
6620 0.9854
6704 0.9863
6747 0.9656
6831 0.9274
6837 0.9887
6887 0.9914
6890 0.9979
6900 0.9448
6959 0.9559
6968 0.9198
6982 0.9426
7006 0.9917
7011 0.9945
7012 0.9918
7050 0.9478
7069 0.9725
7077 0.9262
7120 0.9255
7144 0.9134
7146 0.9307
7166 0.9991
7217 0.9998
7241 0.9161
7251 0.9971
7283 0.9595
7326 0.9434
7336 0.9965
7357 0.9999
7372 0.9183
7412 0.9762
7452 0.9899
7607 0.9028
7610 0.9938
7659 0.9593
7664 0.9308
7689 0.9986
7705 0.9358
7754 0.9993
7786 0.9292
7796 0.9768
7828 0.9985
78

In [ ]:
# Flagmanがいた。

# 6121  4 番目に一応
# 6355  6
# 9006  A
# 12750 I?
# 13622 C
# 15850 4
# 18111 C
# 19059-終わり } 